In [1]:
from socket import * #import the socket module
import time # for RTT
import sys  # forcommandline functions
import datetime
from pathlib import Path

In [ ]:
# Import socket module 
import socket 


def Main(): 
    # local host IP '127.0.0.1' 
    host = '127.0.0.1'

    # Define the port on which you want to connect 
    port = 12345

    s = socket.socket(socket.AF_INET,socket.SOCK_STREAM) 

    # connect to server on local computer 
    s.connect((host,port)) 

    # message you send to server 
    while True: 
        filename = input('\nfile name :') 
        request_type=input('\nrequest type:') 
        # message sent to server 
        s.send((request_type+" "+filename+" "+str(port)).encode('ascii')) 
        if request_type == 'GET':
            f = open(filename, "w")
            while message != '/r/n':
                message =c.recv(1024).decode()
                print(message)
                f.write(message)
                #message =client.recv(1024).decode()
            print('Received from the server :',str(data.decode('ascii'))) 
        elif request_type == 'POST':
            print ('File name is: ', filename)
            my_file = Path(filename)
            if my_file.is_file(): 
                f = open(filename)
                outputdata = f.read()
                s.send("HTTP/1.1 200 OK\r\n\r\n".encode())
                for i in range(0, len(outputdata)):
                    print(outputdata[i])
                    s.send(outputdata[i].encode())
                s.send("\r\n".encode())
                
                
        ans = input('\nDo you want to continue(y/n) :') 
        if ans == 'y': 
            continue
        else: 
            break
    # close the connection 
    s.close() 

if __name__ == '__main__': 
    Main() 




file name :bal7.txt

request type:POST
File name is:  bal7.txt
N
o
u
r
 
w
a
s
 
h
e
r
e


;
d
l
s
k
a
k
f
j
;
l
s
k
d
f
j
d
l
a
;
f
j
l
;
a
s
f
j
l
k
s
d
j
f
l
d
s
j
f
l
;
d
f
s
d
j
l
f
k
l
j
s
l
j
d
f
l
k
j
File name is:  bal7.txt
N
o
u
r
 
w
a
s
 
h
e
r
e


;
d
l
s
k
a
k
f
j
;
l
s
k
d
f
j
d
l
a
;
f
j
l
;
a
s
f
j
l
k
s
d
j
f
l
d
s
j
f
l
;
d
f
s
d
j
l
f
k
l
j
s
l
j
d
f
l
k
j


In [ ]:
my_file = Path("bal7.txt")
if my_file.is_file():
    print('yes')